# Profile NeRF with Timeloop and Accelergy

In [19]:
import os
import re
import yaml

from profiler import Profiler

import sys
sys.path.append("../") # go to parent dir

from accelerating_nerfs.models import VanillaNeRF

In [4]:
# Using vanilla NeRF which are MLPs
model = VanillaNeRF()

# Need to patch the forward method for the purpose of mapping to pass in ray directions
# This ensures the bottleneck layer is captured in the timeloop outputs
model.old_forward = model.forward

def new_forward(self, x):
    return self.old_forward(x, x)

model.forward = new_forward.__get__(model)
print(model)

VanillaNeRF(
  (posi_encoder): SinusoidalEncoder()
  (view_encoder): SinusoidalEncoder()
  (mlp): NerfMLP(
    (base): MLP(
      (hidden_activation): ReLU()
      (output_activation): Identity()
      (hidden_layers): ModuleList(
        (0): Linear(in_features=63, out_features=256, bias=True)
        (1): Linear(in_features=256, out_features=256, bias=True)
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): Linear(in_features=256, out_features=256, bias=True)
        (4): Linear(in_features=256, out_features=256, bias=True)
        (5): Linear(in_features=319, out_features=256, bias=True)
        (6): Linear(in_features=256, out_features=256, bias=True)
        (7): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (sigma_layer): DenseLayer(
      (hidden_activation): ReLU()
      (output_activation): Identity()
      (hidden_layers): ModuleList()
      (output_layer): Linear(in_features=256, out_features=1, bias=True)
    )
    (bottl

In [14]:
profiler = Profiler(
    top_dir='workloads',
    sub_dir='nerf',
    timeloop_dir='simple_weight_stationary',
    model=model,
    input_size=(1, 3),
    batch_size=2 ** 14,
    convert_fc=True,
    exception_module_names=[]
)

results, summary = profiler.profile()
print(summary)

unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'accelerating_nerfs.models.NerfMLP'>
unknown module type <class 'accelerating_nerfs.models.VanillaNeRF'>


running timeloop to get energy and latency...


0it [00:00, ?it/s]

timeloop running finished!
{'total_energy': 421708.64999999997, 'total_cycle': 725352448, 'num_params': 595844, 'macs': 593450, 'activation_size': 2300.0}



/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::unsqueeze". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::reshape". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/usr/local/lib/python3.8/dist-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::sin". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


In [16]:
total_energy = 0
total_cycle = 0

for layer_id, info in results.items():
    print(f"ID: {layer_id} \t Energy: {info['energy']} \t Cycle: {info['cycle']} \t Number of same architecture layers: {info['num']}")
    total_energy += info['energy'] * info['num']
    total_cycle += info['cycle'] * info['num']
    
print(f'\nTotal Energy: {total_energy} uj \nTotal Cycles: {total_cycle}')

ID: 1 	 Energy: 5172.87 	 Cycle: 7340032 	 Number of same architecture layers: 1
ID: 2 	 Energy: 47639.28 	 Cycle: 33554432 	 Number of same architecture layers: 7
ID: 6 	 Energy: 52584.33 	 Cycle: 334495744 	 Number of same architecture layers: 1
ID: 9 	 Energy: 579.42 	 Cycle: 131072 	 Number of same architecture layers: 1
ID: 11 	 Energy: 29561.67 	 Cycle: 148373504 	 Number of same architecture layers: 1
ID: 12 	 Energy: 335.4 	 Cycle: 131072 	 Number of same architecture layers: 1

Total Energy: 421708.64999999997 uj 
Total Cycles: 725352448


In [20]:
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

nerf_layer_dir = "workloads/nerf"
for layer_path in natural_sort(os.listdir(nerf_layer_dir)):
    layer_path = os.path.join(nerf_layer_dir, layer_path)
    
    with open(layer_path, "r") as f:
        layer_config = yaml.safe_load(f)
        
    C = layer_config['problem']['instance']['C']
    M = layer_config['problem']['instance']['M']
    N = layer_config['problem']['instance']['N']
    print(f"{os.path.basename(layer_path)}, C={C}, M={M}, N={N}")

nerf_layer1.yaml, C=63, M=256, N=16384
nerf_layer2.yaml, C=256, M=256, N=16384
nerf_layer3.yaml, C=256, M=256, N=16384
nerf_layer4.yaml, C=256, M=256, N=16384
nerf_layer5.yaml, C=256, M=256, N=16384
nerf_layer6.yaml, C=319, M=256, N=16384
nerf_layer7.yaml, C=256, M=256, N=16384
nerf_layer8.yaml, C=256, M=256, N=16384
nerf_layer9.yaml, C=256, M=1, N=16384
nerf_layer10.yaml, C=256, M=256, N=16384
nerf_layer11.yaml, C=283, M=128, N=16384
nerf_layer12.yaml, C=128, M=3, N=16384
